# Multiclassification with Multi-Output Model
[Download data](https://www.kaggle.com/c/plant-pathology-2021-fgvc8)


Can you help detect farmers detect apple diseases? This competition builds on last year's by challenging you to handle additional diseases and to provide more detailed information about leaves that have multiple infections.

Files
train.csv - the training set metadata.

image - the image ID.

labels - the target classes, a space delimited list of all diseases found in the image. Unhealthy leaves with too many diseases to classify visually will have the complex class, and may also have a subset of the diseases identified.

sample_submission.csv - A sample submission file in the correct format.

- image

- labels

train_images - The training set images.

test_images - The test set images. This competition has a hidden test set: only three images are provided here as samples while the remaining 5,000 images will be available to your notebook once it is submitted.

In [1]:
import os
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras import Model
from keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input,Dense, Activation, Flatten, Dropout, BatchNormalization, MaxPool2D, GlobalMaxPool2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D


In [2]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [3]:
df.head(2)

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex


In [4]:
df.labels.value_counts()

scab                               4826
healthy                            4624
frog_eye_leaf_spot                 3181
rust                               1860
complex                            1602
powdery_mildew                     1184
scab frog_eye_leaf_spot             686
scab frog_eye_leaf_spot complex     200
frog_eye_leaf_spot complex          165
rust frog_eye_leaf_spot             120
rust complex                         97
powdery_mildew complex               87
Name: labels, dtype: int64

In [5]:
df['labels'] = df['labels'].apply(lambda string: string.split(' '))
df

,image,labels
0,800113bb65efe69e.jpg,[healthy]
1,8002cb321f8bfcdf.jpg,"[scab, frog_eye_leaf_spot, complex]"
2,80070f7fb5e2ccaa.jpg,[scab]
3,80077517781fb94f.jpg,[scab]
4,800cbf0ff87721f8.jpg,[complex]
...,...,...
18627,fffb900a92289a33.jpg,[healthy]
18628,fffc488fa4c0e80c.jpg,[scab]
18629,fffc94e092a59086.jpg,[rust]
18630,fffe105cf6808292.jpg,"[scab, frog_eye_leaf_spot]"


In [6]:
X_train,X_valid, train, valid = train_test_split(df['image'],df.drop('image',axis=1),random_state=42,test_size=0.2)
train['image'] = X_train.copy()
valid['image'] = X_valid.copy()

In [7]:
num_class = 6

In [8]:
train.reset_index(drop=True,inplace=True)
valid.reset_index(drop=True,inplace=True)

In [9]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                            preprocessing_function=tf.keras.applications.xception.preprocess_input,
                            rotation_range=45,
                            zoom_range=[-0.1,0.1],
                            shear_range=0.05,
                            horizontal_flip=True,
                            vertical_flip=True
                            )

train_data = datagen.flow_from_dataframe(
    train,
    directory= '../input/plant-pathology-2021-fgvc8/train_images',
    x_col="image",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (150,150),
    class_mode="categorical",
    classes=['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab'],
    batch_size=16,
    shuffle=True,
    seed=40,
    )

Found 14905 validated image filenames belonging to 6 classes.


In [10]:
datagen_valid = tf.keras.preprocessing.image.ImageDataGenerator(
                                                        preprocessing_function=tf.keras.applications.xception.preprocess_input,
                                                        data_format=None,
                                                    )

valid_data = datagen_valid.flow_from_dataframe(
    valid,
    directory= '../input/plant-pathology-2021-fgvc8/train_images',
    x_col="image",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (150,150),
    class_mode="categorical",
    classes=['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab'],
    batch_size=16,
    shuffle=False,
    seed=40,
)

Found 3727 validated image filenames belonging to 6 classes.


In [ ]:
test = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
test2 = test.copy()

In [12]:

datagen_test = tf.keras.preprocessing.image.ImageDataGenerator(
                                                        preprocessing_function=tf.keras.applications.xception.preprocess_input,
                                                        data_format=None,
                                                    )

test_data = datagen_test.flow_from_dataframe(
    test2,
    directory= '../input/plant-pathology-2021-fgvc8/test_images',
    x_col="image",
    color_mode="rgb",
    target_size = (150,150),
    class_mode=None,
    batch_size=1,
    shuffle=False,
)

Found 3 validated image filenames.


In [13]:
# Building our model!
weight_path='../input/tf-keras-pretrained-model-weights/No Top/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model2=Xception(weights=weight_path,include_top=False, input_shape=(150,150,3))
x2=base_model2.output
x2=GlobalAveragePooling2D()(x2)
x2=Dense(128,activation='relu')(x2)
x2=Dropout(0.2)(x2)
x2=Dense(64,activation='relu')(x2)

 
output1 = Dense(1, activation = 'sigmoid')(x2)
output2 = Dense(1, activation = 'sigmoid')(x2)
output3 = Dense(1, activation = 'sigmoid')(x2)
output4 = Dense(1, activation = 'sigmoid')(x2)
output5 = Dense(1, activation = 'sigmoid')(x2)
output6 = Dense(1, activation = 'sigmoid')(x2)

model_2 = Model(inputs=base_model2.input,outputs=[output1,output2,output3,output4,output5,output6])
model_2.compile(optimizer='adam',
        loss = ["binary_crossentropy","binary_crossentropy","binary_crossentropy",
                "binary_crossentropy","binary_crossentropy","binary_crossentropy"],
        metrics = ["accuracy"])

for layer in base_model2.layers:
    layer.trainable=False

In [14]:
#model_2.summary()

In [15]:
#Our model Plot Model
dot_img_file = 'model_2.png'
#tf.keras.utils.plot_model(model_2, to_file=dot_img_file, show_shapes=True)

In [16]:
### Reseting our generators
train_data.reset()
valid_data.reset()

monitor2 = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=4, verbose=1, mode='auto',
        restore_best_weights=True)
history = model_2.fit(
    train_data,
    steps_per_epoch=train_data.samples//train_data.batch_size,
    epochs=1,
    validation_data=valid_data, validation_steps=valid_data.samples//valid_data.batch_size,
    verbose=1,callbacks=[monitor2]
)

931/931 [==============================] - 2940s 3s/step - loss: 2.8913 - dense_2_loss: 0.4802 - dense_3_loss: 0.4819 - dense_4_loss: 0.4833 - dense_5_loss: 0.4830 - dense_6_loss: 0.4810 - dense_7_loss: 0.4819 - dense_2_accuracy: 0.8199 - dense_3_accuracy: 0.8168 - dense_4_accuracy: 0.8140 - dense_5_accuracy: 0.8150 - dense_6_accuracy: 0.8190 - dense_7_accuracy: 0.8189 - val_loss: 2.8358 - val_dense_2_loss: 0.4730 - val_dense_3_loss: 0.4728 - val_dense_4_loss: 0.4725 - val_dense_5_loss: 0.4723 - val_dense_6_loss: 0.4725 - val_dense_7_loss: 0.4727 - val_dense_2_accuracy: 0.8201 - val_dense_3_accuracy: 0.8201 - val_dense_4_accuracy: 0.8201 - val_dense_5_accuracy: 0.8201 - val_dense_6_accuracy: 0.8201 - val_dense_7_accuracy: 0.8201


In [17]:
model_2.save('multiple_output.h5')

In [ ]:
model_2 = tf.keras.models.load_model('multiple_output.h5')

In [ ]:
test_data.reset()
predictions = model_2.predict(test_data)
predictions = np.array(predictions) 
predictions = predictions.reshape(predictions.shape[1],predictions.shape[0])

In [ ]:
class_idx=[]
for pred in predictions:
    pred=list(pred)
    temp=[]
    for i in pred:
        if (i>0.4):#threshold
            temp.append(pred.index(i))
    if (temp!=[]):
        class_idx.append(temp)
    else:
        temp.append(np.argmax(pred))#maybe change here
        class_idx.append(temp)
print(class_idx)

class_dict = train_data.class_indices
def get_key(val):
    for key,value in class_dict.items():
        if (val==value):
            return key
print(class_dict)

sub_pred=[]
for img_ in class_idx:
    img_pred=[]
    for i in img_:
        img_pred.append(get_key(i))
    sub_pred.append( ' '.join(img_pred))
print(sub_pred)

sub2 = test2[['image']]
sub2['labels']=sub_pred
sub2
sub2.to_csv('submission.csv',index=False)